In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from datetime import datetime

locals_df = pd.read_csv('/content/drive/MyDrive/Cogoport dumps/local_rates 7th oct.csv',low_memory = False)
shipments = pd.read_csv('/content/drive/MyDrive/Cogoport dumps/Shipment port New (1) (1).csv',low_memory = False)

## Cleaning

In [2]:
# @title Sell prices for l

sell = locals_df[locals_df['rate_type']=='cogo_assured'].copy()

sell['commodity'] = sell['commodity'].fillna('general')
sell['main_port'] = sell['main_port'].fillna(sell['port'])

# making port_pair column
sell['port_pair'] = sell['port'] + '_' + sell['main_port']
columns = sell.columns.tolist()
main_port_index = columns.index('main_port')
columns.insert(main_port_index + 1, columns.pop(columns.index('port_pair')))
sell=sell[columns].copy()

# Filling empty country code column
sell['extracted_country_code'] = sell['main_port'].str[:2]
sell['country_code'] = sell['country_code'].fillna(sell['extracted_country_code'])
sell.drop(columns=['extracted_country_code'],inplace = True)

# renaming as origin and destination
sell['trade_type'] = sell['trade_type'].replace({'export': 'origin', 'import': 'destination'})
duplicates_sell = sell[sell.duplicated()]
# duplicate_rows
sell.drop_duplicates(inplace= True)
# drop unnnecessary columns
sell.drop(columns=['service_provider','shipping_line','rate_type'],inplace = True)

sell = sell[sell['importer_exporter'].isna()].copy()
sell.drop(columns=['importer_exporter'], inplace=True)

### price in inr
conversion_rates = {
    'INR': 1.0,'USD': 84.5,'GBP': 108, 'SEK': 8, 'ZAR': 5, 'EUR': 91, 'CNY': 12, 'JPY': 0.60,'BRL': 15,'VND': 0.004,
    'MYR': 18.2, 'AED': 23, 'IQD': 0.065, 'THB': 2.5, 'AUD': 55, 'IDR': 0.0051,
    'SGD': 63, 'SAR': 23,'HKD': 11, 'QAR': 23, 'KRW': 0.060, 'NZD': 50, 'OMR': 218, 'GHS': 5.5, 'IRR': 0.002, 'EGP': 1.8, 'MAD': 8.5,
    'TWD': 2.6, 'BHD': 223, 'KWD': 274, 'NOK': 8, 'PKR': 0.3, 'PHP': 1.5, 'NGN': 0.052, 'BDT': 0.71, 'JOD': 118,
    'DKK': 12.15, 'LKR': 0.28, 'PEN': 22.40, 'XOF': 0.14, 'YER': 0.33, 'XAF': 0.14, 'MUR': 1.8, 'VEF': 0.000023, 'BND': 62.5,
    'AOA': 0.1, 'KES': 0.64, 'DJF': 0.47, 'TRY': 2.53, 'GNF': 0.0097, 'SLL': 0.0038, 'SDG': 0.14, 'TND': 26.84, 'JMD': 0.53,'ERN': 5.6, 'CAD': 60.5
}
sell['conversion_rate'] = sell['currency'].map(conversion_rates)
sell['price_in_inr'] = sell['price'] * sell['conversion_rate']
sell.drop(columns=['conversion_rate','price','currency'], inplace=True)

## changing unit column to charge code names
sell.loc[
    (sell['code'] == 'OFDK'),
    'unit'
] = 'OFDK'

sell.loc[
    (sell['code'] == 'AMS'),
    'unit'
] = 'AMS'

sell.loc[
    (sell['unit'] == 'per_shipment'),
    'unit'
] = 'per_bl'

## total price
sell['key1'] = sell['trade_type'] + '_' + sell['port_pair'] + '_' + sell['container_size'].astype(str) + '_' + sell['container_type'] + '_' + sell['commodity']+ '_' + sell['updated_at'].astype(str)
# price by unit
pivot_df = sell.pivot_table(
    index='key1',
    columns='unit',
    values='price_in_inr',
    aggfunc='sum',
    fill_value=0
)
pivot_df.columns = [f'total_price_{unit}' for unit in pivot_df.columns]

# Merge the pivoted columns back into the original DataFrame, based on key1
sell.drop(columns=['unit', 'price_in_inr'], inplace=True)
sell.drop_duplicates(inplace=True)
sell = sell.merge(pivot_df, on='key1')

## latest updated rate
sell['key'] = sell['trade_type'] + '_' + sell['port_pair'] + '_' + sell['container_size'].astype(str) + '_' + sell['container_type'] + '_' + sell['commodity']
sell.drop(columns=['code'], inplace=True)
sell.drop_duplicates(inplace= True)

sell['updated_at'] = pd.to_datetime(sell['updated_at']).dt.date

sell = sell.groupby(['key','country_code','trade_type','port','main_port','port_pair','container_size','container_type','commodity']).agg({
    'total_price_AMS': 'mean',
    'total_price_OFDK': 'mean',
    'total_price_per_bl': 'mean',
    'total_price_per_container': 'mean',
    'updated_at': 'max'
}).reset_index()


sell

,key,country_code,trade_type,port,main_port,port_pair,container_size,container_type,commodity,total_price_AMS,total_price_OFDK,total_price_per_bl,total_price_per_container,updated_at
0,destination_AEAAN_AEAAN_40HC_standard_general,AE,destination,AEAAN,AEAAN,AEAAN_AEAAN,40HC,standard,general,0.0,0.0,11040.0,40250.0,2024-06-19
1,destination_AEAAN_AEAAN_40_standard_general,AE,destination,AEAAN,AEAAN,AEAAN_AEAAN,40,standard,general,0.0,0.0,11040.0,40250.0,2024-06-19
2,destination_AEABU_AEABU_40HC_standard_general,AE,destination,AEABU,AEABU,AEABU_AEABU,40HC,standard,general,0.0,0.0,11040.0,40250.0,2024-06-19
3,destination_AEABU_AEABU_40_standard_general,AE,destination,AEABU,AEABU,AEABU_AEABU,40,standard,general,0.0,0.0,11040.0,40250.0,2024-06-19
4,destination_AEAUH_AEAUH_40HC_refer_general,AE,destination,AEAUH,AEAUH,AEAUH_AEAUH,40HC,refer,general,0.0,0.0,11385.0,31625.0,2024-06-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42887,origin_ZAZDM_ZAZDM_40HC_standard_general,ZA,origin,ZAZDM,ZAZDM,ZAZDM_ZAZDM,40HC,standard,general,0.0,0.0,5825.0,26425.0,2024-06-19
42888,origin_ZAZDM_ZAZDM_40_standard_general,ZA,origin,ZAZDM,ZAZDM,ZAZDM_ZAZDM,40,standard,general,0.0,0.0,5825.0,26425.0,2024-06-19
42889,origin_sESFS_sESFS_20_standard_general,SE,origin,sESFS,sESFS,sESFS_sESFS,20,standard,general,0.0,0.0,5440.0,16320.0,2024-06-19
42890,origin_sESFS_sESFS_40HC_standard_general,SE,origin,sESFS,sESFS,sESFS_sESFS,40HC,standard,general,0.0,0.0,5440.0,16560.0,2024-06-19


In [3]:
# @title Locals Cleaning

locals_df['commodity'] = locals_df['commodity'].fillna('general')
locals_df['main_port'] = locals_df['main_port'].fillna(locals_df['port'])

# making port_pair column
locals_df['port_pair'] = locals_df['port'] + '_' + locals_df['main_port']
columns = locals_df.columns.tolist()
main_port_index = columns.index('main_port')
columns.insert(main_port_index + 1, columns.pop(columns.index('port_pair')))
locals_df=locals_df[columns].copy()

# Filling empty country code column
locals_df['extracted_country_code'] = locals_df['main_port'].str[:2]
locals_df['country_code'] = locals_df['country_code'].fillna(locals_df['extracted_country_code'])
locals_df.drop(columns=['extracted_country_code'],inplace = True)

# renaming as origin and destination
locals_df['trade_type'] = locals_df['trade_type'].replace({'export': 'origin', 'import': 'destination'})

# forwarder
locals_df['forwarder'] = locals_df['service_provider']+'_'+ locals_df['shipping_line']

# correcting a outlier
locals_df.loc[
    (locals_df['port_pair'] == 'INMUN_INMUN') &
    (locals_df['commodity'] == 'general') &
    (locals_df['container_size'] == '20') &
    (locals_df['container_type'] == 'standard') &
    (locals_df['trade_type'] == 'origin')&
    (locals_df['forwarder'] == 'MAERSK LINE A/S_Maersk')&
    (locals_df['code'] == 'MISC')&
    (locals_df['updated_at'] == '2024-08-22 07:21:19.353'),
    'price'
] = 40


# duplicate_rows
duplicate_rows = locals_df[locals_df.duplicated()]
locals_df.drop_duplicates(inplace= True)

# remove customer specific reverts
locals_df = locals_df[locals_df['importer_exporter'].isna()].copy()
locals_df.drop(columns=['importer_exporter'], inplace=True)

locals_df

,id,country_code,trade_type,port,main_port,port_pair,container_size,container_type,commodity,service_provider,shipping_line,code,unit,price,currency,rate_type,updated_at,condition_applied,forwarder
0,0000147d-5532-4b34-8e46-b5f59f5a86c2,IN,destination,INBLR,INKAT,INBLR_INKAT,40HC,standard,general,"SHANGHAI SOUTH LOGISTICS CO.,LTD",Zim,CCI,per_container,2600.0,INR,market_place,2024-01-16 12:24:17.20342,No,"SHANGHAI SOUTH LOGISTICS CO.,LTD_Zim"
1,0000147d-5532-4b34-8e46-b5f59f5a86c2,IN,destination,INBLR,INKAT,INBLR_INKAT,40HC,standard,general,"SHANGHAI SOUTH LOGISTICS CO.,LTD",Zim,DO,per_bl,7000.0,INR,market_place,2024-01-16 12:24:17.20342,No,"SHANGHAI SOUTH LOGISTICS CO.,LTD_Zim"
2,0000147d-5532-4b34-8e46-b5f59f5a86c2,IN,destination,INBLR,INKAT,INBLR_INKAT,40HC,standard,general,"SHANGHAI SOUTH LOGISTICS CO.,LTD",Zim,FCG,per_container,2600.0,INR,market_place,2024-01-16 12:24:17.20342,No,"SHANGHAI SOUTH LOGISTICS CO.,LTD_Zim"
3,0000147d-5532-4b34-8e46-b5f59f5a86c2,IN,destination,INBLR,INKAT,INBLR_INKAT,40HC,standard,general,"SHANGHAI SOUTH LOGISTICS CO.,LTD",Zim,IMP,per_container,1950.0,INR,market_place,2024-01-16 12:24:17.20342,No,"SHANGHAI SOUTH LOGISTICS CO.,LTD_Zim"
4,0000147d-5532-4b34-8e46-b5f59f5a86c2,IN,destination,INBLR,INKAT,INBLR_INKAT,40HC,standard,general,"SHANGHAI SOUTH LOGISTICS CO.,LTD",Zim,ISC,per_container,1600.0,INR,market_place,2024-01-16 12:24:17.20342,No,"SHANGHAI SOUTH LOGISTICS CO.,LTD_Zim"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4082092,ffffb11b-8472-434d-8c75-ed5dfcef8514,IN,destination,INBRC,INNSA,INBRC_INNSA,20,standard,imo_classes-5.1,CogoXpress,CMA CGM,DO,per_bl,7250.0,INR,market_place,2024-06-14 11:44:16.733325,No,CogoXpress_CMA CGM
4082093,ffffb11b-8472-434d-8c75-ed5dfcef8514,IN,destination,INBRC,INNSA,INBRC_INNSA,20,standard,imo_classes-5.1,CogoXpress,CMA CGM,HBL,per_bl,3200.0,INR,market_place,2024-06-14 11:44:16.733325,No,CogoXpress_CMA CGM
4082094,ffffb11b-8472-434d-8c75-ed5dfcef8514,IN,destination,INBRC,INNSA,INBRC_INNSA,20,standard,imo_classes-5.1,CogoXpress,CMA CGM,ISC,per_container,700.0,INR,market_place,2024-06-14 11:44:16.733325,No,CogoXpress_CMA CGM
4082095,ffffb11b-8472-434d-8c75-ed5dfcef8514,IN,destination,INBRC,INNSA,INBRC_INNSA,20,standard,imo_classes-5.1,CogoXpress,CMA CGM,POCH,per_container,1300.0,INR,market_place,2024-06-14 11:44:16.733325,No,CogoXpress_CMA CGM


In [4]:
# @title remove chinese sp at destination india

# drop these service providers for destination
service_providers_to_drop = ['B&C INTERNATIONAL LOGISTICS LTD',
                             'SHANGHAI SOUTH-LOGISTICS CO., LTD','Shenzhen Ocean Star International Transportation Co.,Ltd','Sinoocean Group Ltd.','WINCO LOGISTICS CO. LTD','V-TRANS LOGISTICS CO.,LTD','CHINA OCEAN SHIPPING AGENCY WUHU CO.,LTD','WORLDEX INTERNATIONAL LOGISTICS QINGDAO CO.,LTD.',
                             'Yess Intelligent Logistics(Shanghai) Co., Ltd.','UNITEX INTERNATIONAL FORWARDING (GUANGZHOU)LIMITED QINGDAO BRANCH','SHENZHEN CIMC','CHINA ELEC TRANS GUANGDONG COMPANY','SHENZHEN BA-SHI YUEXIN LOGISTICS DEVELOPMENT.CO.LTD','Jade Freight Co., Ltd.','CTS INTERNATIONAL LOGISTICS CORPORATION LIMITED',
                             'Worldwide Logistics Co. Ltd.','Citi Freight Logistics Inc','AVANA GLOBAL FZCO','MELODY LOGISTICS COMPANY LIMITED','NAM SUNG SHIPPING VIETNAM COMPANY LIMITED','Ocean Network Express (Singapore) Pte. Ltd.',
                             'SHANGHAI SOUTH-LOGISTICS CO., LTD',
                             'Wheel-done (Shenzhen) Logistics Co.,Ltd.','HISUN INT\'L LOGISTICS CO., LTD.','DK INTERNATIONAL LOGISTICS (NINGBO) CO.,LTD.',
                             'LONG SAIL INTERNATIONAL','P.D.T Logistics Co.,Ltd.','MAERSK SINGAPORE PTE. LTD.','awot global logistics (Korea) Co. Ltd','PT. Samudera Caraka Jasa'
                             ]
# Filter out the rows where trade_type is 'destination' and service_provider is in the specified list
locals_df = locals_df[~((locals_df['trade_type'] == 'destination') & (locals_df['country_code']=='IN') &(locals_df['service_provider'].isin(service_providers_to_drop)))]


# remove service provider cogo expresss rates
locals_df = locals_df[locals_df['service_provider'] != 'CogoXpress'].copy()

# remove service provider cogo frieght rates
locals_df = locals_df[locals_df['service_provider'] != 'COGO FREIGHT PRIVATE LIMITED'].copy()

# remove shipping line cogo line rates
locals_df = locals_df[locals_df['shipping_line'] != 'COGO LINE'].copy()

In [5]:
locals_df['rate_type'].value_counts()

,count
rate_type,
market_place,1286783


In [6]:
# @title price calculation
conversion_rates = {
    'INR': 1.0,'USD': 84.5,'GBP': 108, 'SEK': 8, 'ZAR': 5, 'EUR': 91, 'CNY': 12, 'JPY': 0.60,'BRL': 15,'VND': 0.004,
    'MYR': 18.2, 'AED': 23, 'IQD': 0.065, 'THB': 2.5, 'AUD': 55, 'IDR': 0.0051,
    'SGD': 63, 'SAR': 23,'HKD': 11, 'QAR': 23, 'KRW': 0.060, 'NZD': 50, 'OMR': 218, 'GHS': 5.5, 'IRR': 0.002, 'EGP': 1.8, 'MAD': 8.5,
    'TWD': 2.6, 'BHD': 223, 'KWD': 274, 'NOK': 8, 'PKR': 0.3, 'PHP': 1.5, 'NGN': 0.052, 'BDT': 0.71, 'JOD': 118,
    'DKK': 12.15, 'LKR': 0.28, 'PEN': 22.40, 'XOF': 0.14, 'YER': 0.33, 'XAF': 0.14, 'MUR': 1.8, 'VEF': 0.000023, 'BND': 62.5,
    'AOA': 0.1, 'KES': 0.64, 'DJF': 0.47, 'TRY': 2.53, 'GNF': 0.0097, 'SLL': 0.0038, 'SDG': 0.14, 'TND': 26.84, 'JMD': 0.53,'ERN': 5.6, 'CAD': 60.5
}

### price in inr
locals_df['conversion_rate'] = locals_df['currency'].map(conversion_rates)
locals_df['price_in_inr'] = locals_df['price'] * locals_df['conversion_rate']
locals_df.drop(columns=['conversion_rate','price','currency'], inplace=True)

## changing unit column to charge code names
locals_df.loc[
    (locals_df['code'] == 'OFDK'),
    'unit'
] = 'OFDK'

locals_df.loc[
    (locals_df['code'] == 'AMS'),
    'unit'
] = 'AMS'

locals_df.loc[
    (locals_df['unit'] == 'per_shipment'),
    'unit'
] = 'per_bl'

## total price
locals_df['key1'] = locals_df['trade_type'] + '_' + locals_df['port_pair'] + '_' + locals_df['container_size'].astype(str) + '_' + locals_df['container_type'] + '_' + locals_df['commodity'] + '_' + locals_df['shipping_line'] + '_' + locals_df['service_provider'] + '_' + locals_df['updated_at'].astype(str)



# price by unit
pivot_df = locals_df.pivot_table(
    index='key1',
    columns='unit',
    values='price_in_inr',
    aggfunc='sum',
    fill_value=0
)
pivot_df.columns = [f'total_price_{unit}' for unit in pivot_df.columns]

# Merge the pivoted columns back into the original DataFrame, based on key1
locals_df.drop(columns=['unit', 'price_in_inr'], inplace=True)
locals_df.drop_duplicates(inplace=True)
locals_df = locals_df.merge(pivot_df, on='key1')


## latest updated rate
locals_df['key'] = locals_df['trade_type'] + '_' + locals_df['port_pair'] + '_' + locals_df['container_size'].astype(str) + '_' + locals_df['container_type'] + '_' + locals_df['commodity'] + '_' + locals_df['shipping_line'] + '_' + locals_df['service_provider']
locals_df.drop(columns=['code','rate_type'], inplace=True)
locals_df.drop_duplicates(inplace= True)
locals_df.sort_values(by=['key', 'updated_at'], ascending=[True, False], inplace=True)
locals_df.drop_duplicates(subset='key', keep='first', inplace=True)
locals_df.drop(columns=['key1'], inplace=True)

#updated at
locals_df['updated_at'] = pd.to_datetime(locals_df['updated_at'])
locals_df['updated_at'] = locals_df['updated_at'].dt.date

# Define the cutoff date
from datetime import date
cutoff_date = date(2024, 6, 1)

locals_df = locals_df[locals_df['updated_at'] >= cutoff_date].copy()
locals_df.replace(0.0, np.nan, inplace=True)
locals_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146512 entries, 921615 to 83673
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         146512 non-null  object 
 1   country_code               146512 non-null  object 
 2   trade_type                 146512 non-null  object 
 3   port                       146512 non-null  object 
 4   main_port                  146512 non-null  object 
 5   port_pair                  146512 non-null  object 
 6   container_size             146512 non-null  object 
 7   container_type             146512 non-null  object 
 8   commodity                  146512 non-null  object 
 9   service_provider           146512 non-null  object 
 10  shipping_line              146512 non-null  object 
 11  updated_at                 146512 non-null  object 
 12  condition_applied          146512 non-null  object 
 13  forwarder                  146

In [7]:
# @title Shipments cleaning

# remove service provider col
shipments.drop(columns =['service_provider'],inplace = True)

# pol and pod
shipments['origin_main_port'] = shipments['origin_main_port'].fillna(shipments['pol'])
shipments['origin_main_port_code'] = shipments['origin_main_port_code'].fillna(shipments['pol_port_code'])
shipments.dropna(subset=['pol'],inplace = True)

shipments['destination_main_port'] = shipments['destination_main_port'].fillna(shipments['pod'])
shipments['destination_main_port_code'] = shipments['destination_main_port_code'].fillna(shipments['pod_port_code'])
shipments.dropna(subset=['pod'],inplace = True)

#pol port pair
shipments['pol_port_pair'] = shipments['pol_port_code'] + '_' + shipments['origin_main_port_code']
columns = shipments.columns.tolist()
main_port_index = columns.index('origin_main_port_code')
columns.insert(main_port_index + 1, columns.pop(columns.index('pol_port_pair')))
shipments=shipments[columns].copy()

#pod port pair
shipments['pod_port_pair'] = shipments['pod_port_code'] + '_' + shipments['destination_main_port_code']
columns = shipments.columns.tolist()
main_port_index = columns.index('destination_main_port_code')
columns.insert(main_port_index + 1, columns.pop(columns.index('pod_port_pair')))
shipments=shipments[columns].copy()

# drop cancelled shipments
shipments.drop(shipments[shipments['state'] == 'cancelled'].index, inplace = True)



######### ORIGIN AND DESTINATION    ##########

origin=shipments.copy()
destination=shipments.copy()

origin.drop(columns=['pod','pol_country','pod_port_code','pod_country', 'destination_main_port','destination_main_port_code','pod_port_pair','booking_note_status','vessel_sailing_status','shipment_type'], inplace= True)

destination.drop(columns=['pol','pod_country','pol_port_code','pol_country', 'origin_main_port','origin_main_port_code','pol_port_pair','booking_note_status','vessel_sailing_status','shipment_type'], inplace = True)


origin.rename(columns={
    'pol':'port',
    'pol_port_code': 'port_code',
    'origin_main_port': 'main_port',
    'origin_main_port_code': 'main_port_code',
    'pol_port_pair': 'port_pair'
},inplace = True)

destination.rename(columns={
    'pod':'port',
    'pod_port_code': 'port_code',
    'destination_main_port': 'main_port',
    'destination_main_port_code': 'main_port_code',
    'pod_port_pair': 'port_pair'
},inplace = True)

origin['trade_type'] = 'origin'
destination['trade_type'] = 'destination'

# concate to a single df
shipments = pd.concat([origin, destination], ignore_index=True)
shipments['country_code'] = shipments['main_port_code'].str[:2]


shipments['key1'] = shipments['trade_type'] + '_' + shipments['port_pair'] + '_' + shipments['container_size'].astype(str) + '_' + shipments['container_type'] + '_' + shipments['commodity'] + '_' + shipments['shipping_line']


today = pd.Timestamp(datetime.now())
shipments['shipment_date'] = pd.to_datetime(shipments['shipment_date'], dayfirst=True)

# age in months
shipments['age'] = (today.year - shipments['shipment_date'].dt.year) * 12 + (today.month - shipments['shipment_date'].dt.month)

# quarter age
shipments['quarter_age'] = (shipments['age'] // 3) + 1

# containers moved
shipments['containers_moved'] = shipments.groupby(['key1', 'quarter_age'])['containers_count'].transform('sum')
# unique sid counts
shipments['sid_counts'] = shipments.groupby(['key1', 'quarter_age'])['sid'].transform('nunique')

shipments['shipment_date'] = pd.to_datetime(shipments['shipment_date'], dayfirst=True)


## latest updated rate
shipments.sort_values(by=['key1','quarter_age','shipment_date'], ascending=[True,True,False],inplace=True)
shipments.drop_duplicates(subset=['key1','quarter_age'], keep='first', inplace = True)

# remove shipping line cogo line rates
shipments = shipments[shipments['shipping_line'] != 'COGO LINE'].copy()

In [8]:
shipments[shipments['key1']=='origin_INMUN_INMUN_20_standard_general_Maersk']

,sid,shipment_date,state,shipment_source,shipping_line,trade_type,port,port_code,main_port,main_port_code,...,commodity,containers_count,container_size,container_type,country_code,key1,age,quarter_age,containers_moved,sid_counts
114,188703,2024-07-10,confirmed_by_importer_exporter,spot_line_booking,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,1,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,3,2,10,7
441,185581,2024-04-29,completed,quotation,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,1,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,6,3,47,15
957,179722,2024-01-19,completed,quotation,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,1,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,9,4,31,16
1767,173343,2023-10-23,completed,quotation,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,2,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,12,5,97,43
3376,162215,2023-07-29,completed,quotation,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,3,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,15,6,197,56
6190,147844,2023-04-27,completed,spot_line_booking,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,5,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,18,7,290,41
8758,135788,2023-01-31,completed,spot_line_booking,Maersk,origin,Mundra,INMUN,Mundra,INMUN,...,general,3,20,standard,IN,origin_INMUN_INMUN_20_standard_general_Maersk,21,8,46,12


## Single table

In [9]:
buy = locals_df.copy()
containers = shipments.copy()
sell_prices = sell.copy()

In [10]:
# @title working

buy.drop(columns=['key'], inplace=True)
containers.drop(columns=['key1'], inplace=True)
sell_prices.drop(columns=['key'], inplace=True)

# Pre-compute relevant data in one go
buy['key'] = buy['trade_type']+ '_' + buy['port_pair'] + '_' + buy['commodity'] + '_' + buy['container_size'] + '_' + buy['container_type'] + '_' + buy['shipping_line']
containers['key'] = containers['trade_type'] + '_' + containers['port_pair'] + '_' + containers['commodity'] + '_' + containers['container_size'] + '_' + containers['container_type'] + '_' + containers['shipping_line']
sell_prices['key'] = sell_prices['trade_type'] + '_' + sell_prices['port_pair'] + '_' + sell_prices['commodity'] + '_' + sell_prices['container_size'] + '_' + sell_prices['container_type']

# Merge buy and containers on the key
combined = buy.merge(containers[['containers_moved', 'sid_counts', 'age', 'quarter_age', 'key']],
                      on='key', how='left')


# buy.drop(columns=['key'], inplace=True)
# containers.drop(columns=['key'], inplace=True)

# make a key similar to sell_prices
combined.drop(columns=['key'], inplace=True)
combined['key'] =combined['trade_type'] + '_' +combined['port_pair'] + '_' + combined['commodity'] + '_' + combined['container_size'] + '_' + combined['container_type']
# Prepare DataFrame for results
# results = []
combined.replace(0.0, np.nan, inplace=True)

combined.dropna(subset=['containers_moved'], inplace=True)

##### seperate df for calculating AMS and OFDK
combined_ams = combined[~combined['total_price_AMS'].isna()].copy()
combined_ofdk = combined[~combined['total_price_OFDK'].isna()].copy()

# aged containers
combined_ams['aged_containers'] = combined_ams['containers_moved'] * (combined_ams['quarter_age']**(-0.5))
combined_ofdk['aged_containers'] = combined_ofdk['containers_moved'] * (combined_ofdk['quarter_age']**(-0.5))

# ratio
combined_ams['ratio'] = combined_ams.groupby('key')['aged_containers'].transform(lambda x: x / x.sum())
combined_ofdk['ratio'] = combined_ofdk.groupby('key')['aged_containers'].transform(lambda x: x / x.sum())

# individual price columns
combined_ams['predicted_price_AMS'] = (combined_ams['total_price_AMS'] * combined_ams['ratio'])
combined_ofdk['predicted_price_OFDK'] = (combined_ofdk['total_price_OFDK'] * combined_ofdk['ratio'])

combined_ams['predicted_price_AMS'] = combined_ams.groupby('key')['predicted_price_AMS'].transform('sum')
combined_ofdk['predicted_price_OFDK'] = combined_ofdk.groupby('key')['predicted_price_OFDK'].transform('sum')

# dropping unnecessary columns
combined_ams.drop(columns=['id','condition_applied','total_price_OFDK','total_price_per_bl',
                           'total_price_per_container','age','country_code','port','main_port',
                           'forwarder'],inplace = True)
combined_ofdk.drop(columns=['id','condition_applied','total_price_AMS','total_price_per_bl',
                           'total_price_per_container','age','country_code','port','main_port',
                            'forwarder'],inplace = True)


#### per_bl and per_container prices
combined['aged_containers'] = combined['containers_moved'] * (combined['quarter_age']**(-0.5))
combined['ratio'] = combined.groupby('key')['aged_containers'].transform(lambda x: x / x.sum())

# combined['predicted_per_bl'] = (combined['total_price_per_bl'] * combined['ratio'])
# combined['predicted_per_container'] = (combined['total_price_per_container'] * combined['ratio'])

# combined['predicted_per_bl'] = combined.groupby('key')['predicted_per_bl'].transform('sum')
# combined['predicted_per_container'] = combined.groupby('key')['predicted_per_container'].transform('sum')

# dropping unnecessary columns
combined.drop(columns=['id','condition_applied','age','port','main_port','forwarder'],inplace = True)

## predicted ams and odfk without duplicates
predicted_ams = combined_ams.copy()
predicted_ofdk = combined_ofdk.copy()

predicted_ams.drop(columns=['service_provider','shipping_line','total_price_AMS',
                            'containers_moved','sid_counts','quarter_age','updated_at',
                            'aged_containers','ratio'],inplace=True)
predicted_ofdk.drop(columns=['service_provider','shipping_line','total_price_OFDK',
                            'containers_moved','sid_counts','quarter_age','updated_at',
                            'aged_containers','ratio'],inplace=True)

predicted_ams.drop_duplicates(inplace=True)
predicted_ofdk.drop_duplicates(inplace=True)


# get predicted AMS and OFDK prices
combined =combined.merge(predicted_ams[['key','predicted_price_AMS']],
                         on ='key',how='left')
combined =combined.merge(predicted_ofdk[['key','predicted_price_OFDK']],
                         on ='key',how='left')

# calculate predicted per_bl and per_container prices
combined['predicted_per_bl'] = (combined['total_price_per_bl'] * combined['ratio'])
combined['predicted_per_container'] = (combined['total_price_per_container'] * combined['ratio'])

combined['predicted_per_bl'] = combined.groupby('key')['predicted_per_bl'].transform('sum')
combined['predicted_per_container'] = combined.groupby('key')['predicted_per_container'].transform('sum')

############# predicted prices #######

predicted_prices = combined.copy()
predicted_prices.drop(columns=['service_provider','shipping_line','total_price_AMS','total_price_OFDK',
                            'total_price_per_bl','total_price_per_container',
                            'containers_moved','sid_counts','quarter_age','updated_at',
                            'aged_containers','ratio'],inplace=True)
predicted_prices.drop_duplicates(inplace=True)

#### combining sell prices
sell_prices.rename(columns={'total_price_AMS': 'sell_price_AMS',
                            'total_price_OFDK': 'sell_price_OFDK',
                            'total_price_per_bl': 'sell_per_bl',
                            'total_price_per_container': 'sell_per_container',
                            'updated_at':'sell_updated_at'}, inplace=True)

sell_prices.drop(columns=['port','main_port'],inplace = True)


df = predicted_prices.merge(sell_prices[['key','sell_price_AMS','sell_price_OFDK','sell_per_bl','sell_per_container',
                                         'sell_updated_at']],
                            on='key', how='left')
df.fillna(0.0, inplace=True)

df['difference_AMS'] = df['sell_price_AMS'] - df['predicted_price_AMS']
df['difference_OFDK'] = df['sell_price_OFDK'] - df['predicted_price_OFDK']
df['difference_per_bl'] = df['sell_per_bl'] - df['predicted_per_bl']
df['difference_per_container'] =df['sell_per_container'] - df['predicted_per_container']




In [20]:
combined

,country_code,trade_type,port_pair,container_size,container_type,commodity,service_provider,shipping_line,updated_at,total_price_AMS,...,containers_moved,sid_counts,quarter_age,key,aged_containers,ratio,predicted_price_AMS,predicted_price_OFDK,predicted_per_bl,predicted_per_container
0,CN,destination,CNDLC_CNDLC,20,standard,general,"Orient Overseas Container Line (China) Co., Ltd",OOCL,2024-08-08,NaN,...,2.0,1.0,6.0,destination_CNDLC_CNDLC_general_20_standard,0.816497,1.000000,NaN,NaN,5400.000000,9360.000000
1,CN,destination,CNQIN_CNQIN,20,standard,general,"Orient Overseas Container Line (China) Co., Ltd",OOCL,2024-08-08,NaN,...,4.0,1.0,6.0,destination_CNQIN_CNQIN_general_20_standard,1.632993,0.812048,NaN,NaN,5400.000000,9360.000000
2,CN,destination,CNQIN_CNQIN,20,standard,general,"Orient Overseas Container Line (China) Co., Ltd",OOCL,2024-08-08,NaN,...,1.0,1.0,7.0,destination_CNQIN_CNQIN_general_20_standard,0.377964,0.187952,NaN,NaN,5400.000000,9360.000000
3,CN,destination,CNQIN_CNQIN,40HC,standard,general,ZIM INTEGRATED SHIPPING SERVICES LIMITED,Zim,2024-08-12,NaN,...,3.0,1.0,5.0,destination_CNQIN_CNQIN_general_40HC_standard,1.341641,0.353889,NaN,NaN,6000.000000,12072.000000
4,CN,destination,CNQIN_CNQIN,40HC,standard,general,ZIM INTEGRATED SHIPPING SERVICES LIMITED,Zim,2024-08-12,NaN,...,6.0,2.0,6.0,destination_CNQIN_CNQIN_general_40HC_standard,2.449490,0.646111,NaN,NaN,6000.000000,12072.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5271,VN,origin,VNSGN_VNSGN,40HC,standard,general,ZIM INTEGRATED SHIPPING SERVICES (VIETNAM) LLC,Zim,2024-06-14,NaN,...,4.0,4.0,5.0,origin_VNSGN_VNSGN_general_40HC_standard,1.788854,0.029532,3380.0,NaN,6205.748643,19423.398832
5272,VN,origin,VNSGN_VNSGN,40HC,standard,general,ZIM INTEGRATED SHIPPING SERVICES (VIETNAM) LLC,Zim,2024-06-14,NaN,...,17.0,1.0,7.0,origin_VNSGN_VNSGN_general_40HC_standard,6.425396,0.106076,3380.0,NaN,6205.748643,19423.398832
5273,VN,origin,VNSGN_VNSGN,40,standard,general,CMA-CGM VIETNAM JOINT STOCK COMPANY,CMA CGM,2024-08-13,NaN,...,1.0,1.0,7.0,origin_VNSGN_VNSGN_general_40_standard,0.377964,0.144591,NaN,NaN,5538.191938,18943.801494
5274,VN,origin,VNSGN_VNSGN,40,standard,general,COSCO SHIPPING LINES VIETNAM,Cosco,2024-08-14,NaN,...,3.0,3.0,5.0,origin_VNSGN_VNSGN_general_40_standard,1.341641,0.513246,NaN,NaN,5538.191938,18943.801494


In [11]:
# @title Result
df

,country_code,trade_type,port_pair,container_size,container_type,commodity,key,predicted_price_AMS,predicted_price_OFDK,predicted_per_bl,predicted_per_container,sell_price_AMS,sell_price_OFDK,sell_per_bl,sell_per_container,sell_updated_at,difference_AMS,difference_OFDK,difference_per_bl,difference_per_container
0,CN,destination,CNDLC_CNDLC,20,standard,general,destination_CNDLC_CNDLC_general_20_standard,0.0,0.0,5400.000000,9360.000000,0.0,0.0,3480.0,11700.0,2024-08-06,0.0,0.0,-1920.000000,2340.000000
1,CN,destination,CNQIN_CNQIN,20,standard,general,destination_CNQIN_CNQIN_general_20_standard,0.0,0.0,5400.000000,9360.000000,0.0,0.0,5760.0,9900.0,2024-08-21,0.0,0.0,360.000000,540.000000
2,CN,destination,CNQIN_CNQIN,40HC,standard,general,destination_CNQIN_CNQIN_general_40HC_standard,0.0,0.0,6000.000000,12072.000000,0.0,0.0,5760.0,16020.0,2024-08-21,0.0,0.0,-240.000000,3948.000000
3,HK,destination,HKHKG_HKHKG,20,standard,general,destination_HKHKG_HKHKG_general_20_standard,0.0,0.0,6600.000000,22550.000000,0.0,0.0,6325.0,21450.0,2024-07-03,0.0,0.0,-275.000000,-1100.000000
4,IN,destination,INAKV_INNSA,20,standard,general,destination_INAKV_INNSA_general_20_standard,0.0,0.0,8500.000000,13740.000000,0.0,0.0,10500.0,20000.0,2024-06-25,0.0,0.0,2000.000000,6260.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,VN,origin,VNHPH_VNHPH,40,standard,general,origin_VNHPH_VNHPH_general_40_standard,3380.0,0.0,6200.132900,17543.591723,3802.5,0.0,3380.0,21125.0,2024-08-09,422.5,0.0,-2820.132900,3581.408277
354,VN,origin,VNSGN_VNSGN,20,standard,general,origin_VNSGN_VNSGN_general_20_standard,2957.5,0.0,5192.553223,12170.349366,0.0,0.0,0.0,0.0,0.0,-2957.5,0.0,-5192.553223,-12170.349366
355,VN,origin,VNSGN_VNSGN,40HC,refer,general,origin_VNSGN_VNSGN_general_40HC_refer,0.0,0.0,3295.500000,21294.000000,0.0,0.0,3380.0,21547.5,2024-10-03,0.0,0.0,84.500000,253.500000
356,VN,origin,VNSGN_VNSGN,40HC,standard,general,origin_VNSGN_VNSGN_general_40HC_standard,3380.0,0.0,6205.748643,19423.398832,0.0,0.0,0.0,0.0,0.0,-3380.0,0.0,-6205.748643,-19423.398832


# Input

In [37]:
trade_type = 'destination'
ports = 'INMUN_INMUN'
con_size = '20'
con_type = 'standard'
comm = 'general'

# Working

In [38]:
input_key = trade_type + '_' + ports+ '_' + comm + '_' + con_size + '_' + con_type
temp = df[df['key']==input_key]

data={
    'Type:': ['Avg Buy','Sell'],
    'Price_AMS:': [temp['predicted_price_AMS'].values[0],temp['sell_price_AMS'].values[0]],
    'Price_OFDK:': [temp['predicted_price_OFDK'].values[0],temp['sell_price_OFDK'].values[0]],
    'Per_bl:': [temp['predicted_per_bl'].values[0],temp['sell_per_bl'].values[0]],
    'Per_container:': [temp['predicted_per_container'].values[0],temp['sell_per_container'].values[0]]
}

result = pd.DataFrame(data)

table_ams = combined_ams[combined_ams['key']==input_key].copy()
table_ofdk = combined_ofdk[combined_ofdk['key']==input_key].copy()
table = combined[combined['key']==input_key].copy()

table.drop(columns=['predicted_price_AMS','predicted_price_OFDK','predicted_per_bl','predicted_per_container'],inplace = True)

ams_sl = table_ams['shipping_line'].unique().tolist()
ofdk_sl = table_ofdk['shipping_line'].unique().tolist()

# Display the DataFrame
# print('Shipping Lines charging AMS:', ams_sl),
# print('Shipping Lines charging OFDK:', ofdk_sl)
# result

# Output

In [39]:
# Display the DataFrame
print('Shipping Lines charging AMS:', ams_sl),
print('Shipping Lines charging OFDK:', ofdk_sl)
result

Shipping Lines charging AMS: ['ASYAD']
Shipping Lines charging OFDK: ['Cosco', 'Emirates', 'Hapag Lloyd', 'RCL', 'Wan Hai']


,Type:,Price_AMS:,Price_OFDK:,Per_bl:,Per_container:
0,Avg Buy,2000.0,2969.308727,9612.491685,16149.278523
1,Sell,0.0,0.000000,0.000000,0.000000


In [40]:
# table

In [41]:
# @title per_bl, per_container plot


def create_pie_chart(df, column):
    df['unique_labels'] = df[column].astype(str) + ' | ' + df.index.astype(str)

    fig = go.Figure(
        go.Pie(
            labels=df['unique_labels'],
            values=df['ratio'],
            hoverinfo='label+value+percent',
            hovertemplate=(
                '<b>%{label}</b><br>' +
                'Ratio: %{value}<br>' +
                'Percentage: %{percent}<br>' +
                'Shipping Line: ' + df['shipping_line'].astype(str) + '<br>' +
                'Service Provider: ' + df['service_provider'].astype(str) + '<br>' +
                'Containers Moved: ' + df['containers_moved'].astype(str) + '<br>' +
                'Quarter Age: ' + df['quarter_age'].astype(str) + '<br>' +
                'Updated At: ' + df['updated_at'].astype(str) + '<extra></extra>'
            ),
            textinfo='none',  # no need to display info
            pull=[0.05 if val < 0.01 else 0 for val in df['ratio']],
        )
    )

    fig.update_layout(title=f"Pie Chart for {column}", showlegend=True, width=500, height=500)
    return fig

# Pie chart for the total_price_per_bl
fig_bl = create_pie_chart(table, 'total_price_per_bl')
fig_bl.show()

# Pie chart for the total_price_per_container
fig_bl = create_pie_chart(table, 'total_price_per_container')
fig_bl.show()

In [42]:
# @title AMS, OFDK plots

def create_pie_chart(df, column):

    df['unique_labels'] = df[column].astype(str) + ' | ' + df.index.astype(str)

    fig = go.Figure(
        go.Pie(
            labels=df['unique_labels'],
            values=df['ratio'],
            hoverinfo='label+value+percent',
            hovertemplate=(
                '<b>%{label}</b><br>' +
                'Ratio: %{value}<br>' +
                'Percentage: %{percent}<br>' +
                'Shipping Line: ' + df['shipping_line'].astype(str) + '<br>' +
                'Service Provider: ' + df['service_provider'].astype(str) + '<br>' +
                'Containers Moved: ' + df['containers_moved'].astype(str) + '<br>' +
                'Quarter Age: ' + df['quarter_age'].astype(str) + '<br>' +
                'Updated At: ' + df['updated_at'].astype(str) + '<extra></extra>'
            ),
            text=df[column].apply(lambda x: str(x)),
            textinfo='label',
            pull=[0.05 if val < 0.01 else 0 for val in df['ratio']],
        )
    )
    fig.update_layout(title=f"Pie Chart for {column}", showlegend=True)
    return fig

# Pie chart for the total_price_AMS
fig_ams = create_pie_chart(table_ams, 'total_price_AMS')
fig_ams.show()

# Pie chart for the total_price_OFDK
fig_ofdk = create_pie_chart(table_ofdk, 'total_price_OFDK')
fig_ofdk.show()